In [1]:
# !pip install requests planet

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import ee, eemont
from forestry_carbon_arr.core import ForestryCarbonARR
from forestry_carbon_arr.utils.zarr_utils import save_dataset_efficient_zarr, load_dataset_zarr

import gcsfs
import os

fs = gcsfs.GCSFileSystem(project=os.getenv("GOOGLE_CLOUD_PROJECT"), token='/usr/src/app/user_id.json')


forestry = ForestryCarbonARR(config_path='./00_input/korindo.json')
forestry.initialize_gee()

✓ GEE Initialized successfully
  Credentials Path: /usr/src/app/user_id.json - loaded successfully


In [4]:
# aoi
from forestry_carbon_arr.core.utils import DataUtils
import geopandas as gpd
import geemap

data_utils = DataUtils(forestry.config, use_gee=True)
aoi_gpd, aoi_ee = data_utils.load_geodataframe_gee(forestry.config["AOI_path"])

aoi_gpd_utm = aoi_gpd.to_crs(epsg=32749)

print(f"✅ AOI loaded: {len(aoi_gpd_utm)} features")
print(f"   Area: {aoi_gpd_utm.geometry.area.sum()/10000:.2f} hectares")

/opt/venv/lib/python3.12/site-packages/geemap/conversion.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


✅ AOI loaded: 1 features
   Area: 144217.67 hectares


In [5]:
## aoi smaller - aoi buffer
import geemap

AOIsmall = ee.FeatureCollection('projects/remote-sensing-476412/assets/korindo_with_buffer')
aoi_gpd_small = geemap.ee_to_gdf(AOIsmall)
# aoi_gpd_small


In [6]:
"""
PlanetScope Image Downloader - Import from forestry_carbon_arr library
"""

# Import directly from planet_api submodules (avoids loading main package __init__.py)
# This works in Docker because forestry_carbon_arr is mounted and in PYTHONPATH
from forestry_carbon_arr.planet_api.downloader import PlanetScopeDownloader
from forestry_carbon_arr.planet_api.workflow import PlanetScopeWorkflow

print("✅ Imported PlanetScopeDownloader and PlanetScopeWorkflow from forestry_carbon_arr.planet_api")


✅ Imported PlanetScopeDownloader and PlanetScopeWorkflow from forestry_carbon_arr.planet_api


In [7]:
import os

GCS_BUCKET_PLANET = os.getenv('GCS_BUCKET_PLANET')

# Initialize
workflow = PlanetScopeWorkflow(api_key=os.getenv('PLANET_API_KEY'),  # or pass directly
                              gcs_project=os.getenv('GOOGLE_CLOUD_PROJECT'))

# Run workflow with visualization (no automatic download)
results = workflow.run_complete_workflow(
    # gcs_shp_path=f'gs://{GCS_BUCKET_PATH}/01-korindo/aoi_buffer/korindo_buffer.shp',
    aoi_gdf = aoi_gpd_small,
    start_date="2024-08-01T00:00:00Z",
    end_date="2024-09-30T23:59:59Z",
    cloud_cover_max=0.1,
    visualize=True,  # Shows map with toggleable date-based mosaics
    create_order=False  # Review first, then create order manually
)

STEP 1: Using provided AOI GeoDataFrame

STEP 2: Converting AOI to GeoJSON and Simplifying
✅ Converted AOI to GeoJSON
   Type: MultiPolygon
   📐 AOI geometry: 5086 vertices
   ✅ Simplified to 801 vertices (tolerance: 0.000100)

STEP 3: Searching PlanetScope Images

🔍 Searching PlanetScope images...
   Date range: 2024-08-01T00:00:00Z to 2024-09-30T23:59:59Z
   Max cloud cover: 10.0%
   Item type: PSScene
🔍 Searching for PlanetScope images...
   Date range: 2024-08-01T00:00:00Z to 2024-09-30T23:59:59Z
   Max cloud cover: 10.0%
✅ Found 66 images

STEP 4: Creating Visualization

🗺️  Creating visualization map...
⚠️  Error simplifying geometry: Unknown geometry type: 'featurecollection'
   Using original geometry (may fail if too complex)
   📅 Creating tile layers for 16 dates...
      ✅ Added: 🌍 Planet Mosaic - 2024-08-01 (1 scenes)
      ✅ Added: 🌍 Planet Mosaic - 2024-08-04 (5 scenes)
      ✅ Added: 🌍 Planet Mosaic - 2024-08-05 (2 scenes)
      ✅ Added: 🌍 Planet Mosaic - 2024-09-01 (1 s

In [8]:
results['map']

In [9]:
# ============================================================================
# Verify GCS Delivery Setup BEFORE Creating Order
# ============================================================================
# This ensures Planet API can deliver files to your GCS bucket

# Step 1: Verify GCS bucket access and permissions
verification = workflow.verify_gcs_delivery(
    gcs_bucket=GCS_BUCKET_PLANET,
    gcs_credentials='./planet_sa_gcs.json',
    gcs_path='korindo/2024-09-03/',
    check_existing=True
)

# Check results
print("\n" + "=" * 60)
print("GCS Delivery Verification Results")
print("=" * 60)
print(f"   Bucket exists: {verification['bucket_exists']}")
print(f"   Can write: {verification['can_write']}")
print(f"   Credentials valid: {verification['credentials_valid']}")
print(f"   Ready for delivery: {verification['ready_for_delivery']}")

if verification['ready_for_delivery']:
    print("\n✅ GCS delivery is ready! You can proceed with order creation.")
else:
    print("\n❌ GCS delivery is NOT ready. Please fix the following:")
    if not verification['bucket_exists']:
        print("   - Bucket does not exist")
    if not verification['can_write']:
        print("   - Cannot write to bucket (check IAM permissions)")
    if not verification['credentials_valid']:
        print("   - Credentials are invalid")
    print("\n   Required GCS IAM permissions for Planet API:")
    print("   - storage.objects.create")
    print("   - storage.objects.get")
    print("\n   Note: A test file will be created during verification to test write permission.")
    print("   The test file will NOT be deleted (delete permission is NOT required).")

# Step 2: Get items grouped by date and select a date for ordering
items = results['items']
items_by_date = workflow.group_items_by_date(items)

# Show available dates
print("\n" + "=" * 60)
print("Available dates with scenes:")
print("=" * 60)
for date_str in sorted(items_by_date.keys()):
    count = len(items_by_date[date_str])
    print(f"  {date_str}: {count} scenes")

# Select a date (you can change this to any date from the list above)
date_str = "2024-09-03"  # Change this to the date you want to order

# Get items for selected date
date_items = items_by_date.get(date_str, [])
if not date_items:
    raise ValueError(f"No items found for date: {date_str}")

# Extract item IDs
date_item_ids = [item['id'] for item in date_items]

print(f"\n📅 Selected date: {date_str}")
print(f"   Item IDs ({len(date_item_ids)}): {date_item_ids[:5]}..." if len(date_item_ids) > 5 else f"   Item IDs: {date_item_ids}")


🔍 Verifying GCS delivery setup for bucket: planet-orders-t
   ✅ Credentials loaded from file: ./planet_sa_gcs.json
   ✅ Bucket exists: planet-orders-t
   🔍 Testing write permission in main directory: korindo/
   ✅ Write permission: OK
   ℹ️  Test file created: korindo/planetverify_1763726635.txt (can be manually deleted if needed)

   🔍 Checking for existing files at: gs://planet-orders-t/korindo/2024-09-03/
   ⚠️  Could not check for existing files: 403 GET https://storage.googleapis.com/storage/v1/b/planet-orders-t/o?maxResults=1&projection=noAcl&prefix=korindo%2F2024-09-03%2F&prettyPrint=false: planet-order@remote-sensing-476412.iam.gserviceaccount.com does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist).

✅ GCS delivery is ready!
   Planet API can deliver files to: gs://planet-orders-t/korindo/2024-09-03/

GCS Delivery Verification Results
   Bucket exists: True
   Can write: True
 

In [10]:
# Step 3: If verification passes, create order with GCS delivery
if verification['ready_for_delivery']:
    print("\n" + "=" * 60)
    print("Creating order with GCS delivery...")
    print("=" * 60)
    
    order = workflow.create_clipped_harmonized_order(
        item_ids=date_item_ids,
        aoi_geojson=results['aoi_geojson'],
        bundle_type='analytic_8b_sr_udm2',
        order_name=f'planet_mosaic_{date_str}',
        gcs_bucket=GCS_BUCKET_PLANET,  # Use the bucket from env var
        gcs_path=f'korindo/{date_str}/',
        gcs_credentials='./planet_sa_gcs.json',  # Add credentials parameter
        verify_gcs=False  # Already verified above, skip duplicate check
    )
    
    order_id = order['id']
    print(f"\n✅ Order created: {order_id}")
    print(f"   Files will be delivered to: korindo/{date_str}/")
else:
    print("\n⚠️  Skipping order creation until GCS is ready")
    print("   Fix the issues above and run verification again")


Creating order with GCS delivery...
   ✅ Credentials loaded from file and encoded to base64
   🔒 Note: Full service account JSON (including private_key) will be sent to Planet API
      This is required for GCS direct delivery. Use a dedicated service account with minimal permissions.
   📋 GCS delivery config: bucket=planet-orders-t, path_prefix=korindo/2024-09-03/, credentials=***
   📐 AOI geometry: 801 vertices
   ✅ Geometry is simple enough (801 <= 1000 vertices)
📦 Creating clipped order: planet_mosaic_2024-09-03
   Items: 13
   Bundle type: analytic_8b_sr_udm2
   Harmonized: True
   ✅ Added clip, harmonize (Sentinel-2), composite, and COG format tools
   ✅ Added GCS delivery: planet-orders-t
✅ Order created: 25ef326f-ac78-4b7a-a71a-0fe25f6fa1b5

✅ Order created: 25ef326f-ac78-4b7a-a71a-0fe25f6fa1b5
   Files will be delivered to: korindo/2024-09-03/


In [14]:
# Step 4: Check order status
status = workflow.check_order_status(order_id)
print(f"\n📊 Order status: {status['state']}")

# Step 5: Monitor order (files will be in GCS automatically when ready)
if status['state'] == 'success':
    print(f"\n✅ Order completed!")
    print(f"   Files available in GCS: {date_str}")
    print(f"   Files are individual GeoTIFFs (not zip) - ready to use!")
else:
    print(f"\n⏳ Order still processing. Check again later:")
    print(f"   status = workflow.check_order_status('{order_id}')")


📊 Order status: success

✅ Order completed!
   Files available in GCS: 2024-09-03
   Files are individual GeoTIFFs (not zip) - ready to use!
